### Guilda IA Zup: scikit-learn e o problema do Titanic 

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

Ler os arquivos: <br> '../input/train.csv' e armazenar no pd.DataFrame 'data_train' <br> '../input/test.csv' e armazenar no pd.DataFrame 'data_test'

In [ ]:
data_train = pd.read_csv('../input/train.csv')
data_test = pd.read_csv('../input/test.csv')

Qual a diferença entre um arquivo de treino e um arquivo de teste?

In [ ]:
data_train.head(1)

#### Engenharia de features

Execute a célula abaixo para preparação dos dados (hoje nosso foco não é aqui!)

In [ ]:
def simplify_ages(df):
    df.Age = df.Age.fillna(-0.5)
    bins = (-1, 0, 5, 12, 18, 25, 35, 60, 120)
    group_names = ['Unknown', 'Baby', 'Child', 'Teenager', 'Student', 'Young Adult', 'Adult', 'Senior']
    categories = pd.cut(df.Age, bins, labels=group_names)
    df.Age = categories
    return df

def simplify_cabins(df):
    df.Cabin = df.Cabin.fillna('N')
    df.Cabin = df.Cabin.apply(lambda x: x[0])
    return df

def simplify_fares(df):
    df.Fare = df.Fare.fillna(-0.5)
    bins = (-1, 0, 8, 15, 31, 1000)
    group_names = ['Unknown', '1_quartile', '2_quartile', '3_quartile', '4_quartile']
    categories = pd.cut(df.Fare, bins, labels=group_names)
    df.Fare = categories
    return df

def format_name(df):
    df['Lname'] = df.Name.apply(lambda x: x.split(' ')[0])
    df['NamePrefix'] = df.Name.apply(lambda x: x.split(' ')[1])
    return df    
    
def drop_features(df):
    return df.drop(['Ticket', 'Name', 'Embarked'], axis=1)

def transform_features(df):
    df = simplify_ages(df)
    df = simplify_cabins(df)
    df = simplify_fares(df)
    df = format_name(df)
    df = drop_features(df)
    return df

data_train = transform_features(data_train)
data_test = transform_features(data_test)
data_train.head()

#### Começando com o scikit-learn

1. Pre-processamento para converter string em número
2. Dividir a base 'data_train' em treino e teste
3. Utilizar um algoritmo de classificação para gerar um modelo
4. Avaliar o modelo gerado
5. Otimizar a escolha dos parâmetros do algoritmo e avaliar o resultado
6. **Homework:** escolher um algoritmo de classificação diferente e gerar um modelo
    * Qual foi o melhor modelo? 
    * Por que?

In [ ]:
## preprocessing
from sklearn import preprocessing

## classification
from sklearn.ensemble import RandomForestClassifier

## model selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

## metrics
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.metrics import classification_report

**1. Pre-processamento pra converter string em numero**
<br> Utilizar o 'preprocessing.LabelEncoder()' para cada feature do seguinte array:
<br> features = ['Fare', 'Cabin', 'Age', 'Sex', 'Lname', 'NamePrefix']
<br> 
le = preprocessing.LabelEncoder() <br>
le = le.fit(df_combined[feature]) <br>
df_train[feature] = le.transform(df_train[feature]) <br>
df_test[feature] = le.transform(df_test[feature]) <br>

In [ ]:
def encode_features(df_train, df_test):
    features = ['Fare', 'Cabin', 'Age', 'Sex', 'Lname', 'NamePrefix']
    df_combined = pd.concat([df_train[features], df_test[features]])
    
    for feature in features:
        le = preprocessing.LabelEncoder()
        le = le.fit(df_combined[feature])
        df_train[feature] = le.transform(df_train[feature])
        df_test[feature] = le.transform(df_test[feature])
    return df_train, df_test
    
data_train, data_test = encode_features(data_train, data_test)
data_train.head()

**2. Dividir a base 'data_train' em treino e teste**
<br> Utilizar 'train_test_split' separando 20% para teste e armazenar nas variaveis
<br> X_train, X_test, y_train, y_test

In [ ]:
X_all = data_train.drop(['Survived', 'PassengerId'], axis=1)
y_all = data_train['Survived']

num_test = 0.20
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=num_test, random_state=23)

** 3. Utilizar um algoritmo de classificação para gerar um modelo **
<br> Utilizar o 'RandomForestClassifier'
<br> para treinar, utilize '.fit(X_train, y_train)'

In [ ]:
clf = RandomForestClassifier(n_estimators=10)
clf.fit(X_train, y_train)

**4. Avaliar o modelo gerado**
<br> para predizer, utilize '.predict(X_test, y_test)'
<br> utilize 'accuracy_score(y_test, predictions)' para obter a acurácia
<br> depois utilize 'classification_report(y_test, predictions)' para obter outras avaliações

In [ ]:
predictions = clf.predict(X_test)
print(accuracy_score(y_test, predictions))
print(classification_report(y_test, predictions))

**5. Otimizar a escolha dos parâmetros do algoritmo e avaliar o resultado**
<br> Utilizaremos o 'GridSearchCV' para gerar um novo modelo
<br> Pausa para explicar o conceito importantíssimo de ** CROSS-VALIDATION **

In [ ]:
parameters = {'n_estimators': [4, 6, 9], 
              'max_features': ['log2', 'sqrt','auto'], 
              'criterion': ['entropy', 'gini'],
              'max_depth': [2, 3, 5, 10], 
              'min_samples_split': [2, 3, 5],
              'min_samples_leaf': [1,5,8]
             }

acc_scorer = make_scorer(accuracy_score)
grid_obj = GridSearchCV(clf, parameters, scoring=acc_scorer, cv=5, iid=False)
grid_obj = grid_obj.fit(X_train, y_train)
clf = grid_obj.best_estimator_
clf.fit(X_train, y_train)

Qual a performance desse modelo? (accuracy_score e classification_report)

In [ ]:
predictions = clf.predict(X_test)
print(accuracy_score(y_test, predictions))
print(classification_report(y_test, predictions))

** 6. Homework **

Vá até a documentação da API que vimos hoje ** scikit-learn ** <br>
e escolha um algoritmo de aprendizado supervisionado e gere um modelo. <br>
(https://scikit-learn.org/stable/supervised_learning.html#supervised-learning) <br>
O quão bom é o seu modelo?